In [1]:
# Pseudo-Label Generator with Contour Fitting (Ellipse Only)
import os
import cv2
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
from model_teacher import TeacherNet, denormalize

# Paths
VIDEO_FRAMES_DIR = "G:/Sajal_Data/Obj_4_Code/TAPNet_Dataset/video_frames"
OUTPUT_MASK_DIR = "G:/Sajal_Data/Obj_4_Code/TAPNet_Dataset/contour_masks"
OUTPUT_CSV_PATH = "G:/Sajal_Data/Obj_4_Code/TAPNet_Dataset/contour_labels.csv"
MIN_MAX_PATH = "biometric_min_max.csv"
MODEL_PATH = "teacher_model.pth"

os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)

# Device and model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TeacherNet().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Transforms
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

label_map = {0: "head", 1: "abdomen", 2: "femur"}

results = []

for img_file in sorted(os.listdir(VIDEO_FRAMES_DIR)):
    if not img_file.lower().endswith(('.jpg', '.png')):
        continue

    img_path = os.path.join(VIDEO_FRAMES_DIR, img_file)
    image = Image.open(img_path)
    orig_size = image.size
    tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        out_cls, out_seg, out_val = model(tensor)
        pred_label = torch.argmax(out_cls, dim=1).item()
        label_str = label_map[pred_label]
        val_norm = out_val.item()
        real_vals = denormalize(val_norm, label_str, MIN_MAX_PATH)

        pred_mask = torch.sigmoid(out_seg).squeeze().cpu().numpy()
        pred_mask = cv2.resize(pred_mask, orig_size[::-1])
        pred_mask = (pred_mask > 0.2).astype(np.uint8) * 255  # lowered threshold to 0.2

    # Find contours
    contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask_canvas = np.zeros_like(pred_mask)

    if len(contours) > 0:
        cnt = max(contours, key=cv2.contourArea)
        if label_str in ["head", "abdomen"] and len(cnt) >= 5:
            ellipse = cv2.fitEllipse(cnt)
            if ellipse[1][0] > 0 and ellipse[1][1] > 0:
                cv2.ellipse(mask_canvas, ellipse, 255, thickness=2)
            else:
                print(f"⚠️ Invalid ellipse dimensions in {img_file}, skipping ellipse draw.")
        elif label_str == "femur" and len(cnt) >= 3:
            # Option 1: Convex hull
            hull = cv2.convexHull(cnt)
            cv2.drawContours(mask_canvas, [hull], -1, 255, thickness=2)

            # Option 2 (alternative): filled contour if you prefer a region mask
            # cv2.drawContours(mask_canvas, [cnt], -1, 255, thickness=cv2.FILLED)
    else:
        print(f"⚠️ No contour found in {img_file}, skipping drawing.")

    # Save mask
    mask_name = img_file.replace(".jpg", "_mask.png").replace(".png", "_mask.png")
    cv2.imwrite(os.path.join(OUTPUT_MASK_DIR, mask_name), mask_canvas)

    for biom_name, val in real_vals.items():
        results.append({"image": img_file, "plane": label_str, "type": biom_name, "value": val})

pd.DataFrame(results).to_csv(OUTPUT_CSV_PATH, index=False)
print("✅ Contour-fitting pseudo-label generation complete.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_12240\2242615945.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=d

⚠️ No contour found in seq_000_frame_0007.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0008.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0009.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0010.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0011.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0013.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0015.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0016.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0017.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0018.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0019.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0022.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0024.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0025.png, skipping drawing.
⚠️ No contour found in seq_000_frame_0028.png, skipping drawing.
⚠️ No contour found in se